# News API

A Python client for the [News API](https://newsapi.org/docs/).

## General

This is a Python client library for [News API V2](https://newsapi.org/).
The functions and methods for this library should mirror the
endpoints specified by the News API [documentation](https://newsapi.org/docs/endpoints).

## Installation

Installation for the package can be done via `pip`:

In [ ]:
%pip install newsapi-python
%pip install opencv-python
%pip install mediapipe
%pip install screeninfo
%pip install cvzone
%pip install matplotlib
%pip install pandas

## Usage

After installation, import the client class into your project:

In [ ]:
from newsapi import NewsApiClient

Initialize the client with your API key:

In [ ]:
import os

api = NewsApiClient(api_key=os.environ['NEWSAPI_ORG'])

### Endpoints

An instance of `NewsApiClient` has three instance methods corresponding to three News API endpoints.

#### Top Headlines

Use `.get_top_headlines()` to pull from the [`/top-headlines`](https://newsapi.org/docs/endpoints/top-headlines) endpoint:

In [ ]:
api.get_top_headlines(sources='bbc-news')

#### Everything

Use `.get_everything()` to pull from the [`/everything`](https://newsapi.org/docs/endpoints/everything) endpoint:

In [ ]:
api.get_everything(q='bitcoin')

#### Sources

Use `.get_sources()` to pull from the [`/sources`](https://newsapi.org/docs/endpoints/sources) endpoint:

In [ ]:
api.get_sources()

## Warming up

Let's get some news but using it together with OpenCV and Pandas


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# init
api = NewsApiClient(api_key=os.environ['NEWSAPI_ORG'])

In [ ]:
def get_top_headlines():
    response = api.get_top_headlines(
        sources='bbc-news, cnn, fox-news, google-news, the-new-york-times, the-wall-street-journal, the-washington-post, time, usa-today, wired'
    )
    return response


top_headlines = get_top_headlines()

In [ ]:
print(
    """
    Top Headlines
    -------------
    {}
    """.format(top_headlines)
)

In [ ]:
def news_to_df(news):
    # map all the articles to a list
    articles = list(map(lambda x: x['title'], news['articles']))

    # create a dataframe
    df = pd.DataFrame(articles, columns=['title'])

    # add a column for the length of the title
    df['title_length'] = df['title'].apply(lambda x: len(x))

    # add a column for the number of words in the title
    df['title_words'] = df['title'].apply(lambda x: len(x.split(' ')))

    return df

In [ ]:
def news_to_df_all_cols(news):
    # map all the articles to a list, including all columns. Only for source column, we will take the name of the source
    articles = list(map(lambda x: [x['title'], x['description'], x['url'], x['urlToImage'], x['publishedAt'], x['source']['name']], news['articles']))

    # set column names
    columns = ['title', 'description', 'url', 'urlToImage', 'publishedAt', 'source']
    
    # create a dataframe
    df = pd.DataFrame(articles, columns=columns)

    return df

In [ ]:
df = news_to_df(top_headlines)
df.head()

In [ ]:
df = news_to_df_all_cols(top_headlines)
df.head()

## News API Client with OpenCV

Simple WebCam Client directly consuming the NEWS API with blocking calls

In [25]:
import cv2
import cvzone
import screeninfo
import mediapipe as mp

# Using OpenCV to display the image
cap = cv2.VideoCapture(0)

cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
cap.set(cv2.CAP_PROP_FPS, 60)

# text as one line string
text = ' '.join(df['title'].tolist())

# add '###' between each title
text = ' ... | '.join(df['title'].tolist())

mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

zoomed_text_color = (0, 255, 0)
standard_text_color = (255, 255, 255)
color = standard_text_color

direction = 0
font_size = 12
box_size = 50

with mp_hands.Hands(
        model_complexity=0,
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5) as hands:

    while cap.isOpened():

        success, image = cap.read()

        if not success:
            print("Ignoring empty camera frame.")
            # If loading a video, use 'break' instead of 'continue'.
            continue

        image = cv2.flip(image, 1)

        # To improve performance, optionally mark the image as not writeable to
        # pass by reference.
        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = hands.process(image)

        # # Draw the hand annotations on the image.
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # create a background for the text with a border
        cv2.rectangle(image, (0, image.shape[0] - box_size), (image.shape[1], image.shape[0]), (0, 0, 0), -1)
        # border
        cv2.rectangle(image, (0, image.shape[0] - box_size), (image.shape[1], image.shape[0]), (255, 255, 255), 2)


        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                # mp_drawing.draw_landmarks(
                #     image,
                #     hand_landmarks,
                #     mp_hands.HAND_CONNECTIONS,
                #     mp_drawing_styles.get_default_hand_landmarks_style(),
                #     mp_drawing_styles.get_default_hand_connections_style())

                # if left hand is raised then move the text to the left
                if hand_landmarks.landmark[mp_hands.HandLandmark.WRIST].x < 0.2 and len(results.multi_hand_landmarks) == 1:
                    text = text[1:] + text[0]
                    direction = 0
                    font_size = 12
                    color = standard_text_color
                    box_size = 50

                # if right hand is raised then move the text to the right
                elif hand_landmarks.landmark[mp_hands.HandLandmark.WRIST].x > 0.8 and len(results.multi_hand_landmarks) == 1:
                    direction = 1
                    font_size = 12
                    color = standard_text_color
                    box_size = 50

                # if both hands are raised then increase the font size to 36pt and change the color
                elif 0.2 < hand_landmarks.landmark[mp_hands.HandLandmark.WRIST].x < 0.8 and len(results.multi_hand_landmarks) == 2:
                    font_size = 36
                    color = zoomed_text_color
                    box_size = 100

        else:
            # if no hands are detected then move the text to the left
            font_size = 12
            color = standard_text_color
            box_size = 50

        if direction == 0:
            text = text[1:] + text[0]
        elif direction == 1:
            text = text[-1] + text[:-1]

        # draw the text
        cv2.putText(image, text, (2, image.shape[0] - 10), cv2.FONT_HERSHEY_SIMPLEX, font_size / 12, color, 2)

        # put a logo in the top left corner
        logo = cv2.imread('logo.png')

        # resize the logo
        logo = cv2.resize(logo, (int(logo.shape[1] / 12), int(logo.shape[0] / 12)))

        # add the logo to the image
        image[0:logo.shape[0], 0:logo.shape[1]] = logo

        # Flip the image horizontally for a selfie-view display.
        cv2.imshow('AI News', image)

        # wait for the 'q' key to be pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

        # if 's' is pressed, save the image
        if cv2.waitKey(1) & 0xFF == ord('s'):
            cv2.imwrite('news.jpg', image)

cap.release()

# stop the video
cv2.destroyAllWindows()


KeyboardInterrupt: 